# Git settings

In [107]:
%cd drive/My\ Drive/Colab\ Notebooks/DLG/slack_report/slack-report-2020

[Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks/DLG/slack_report/slack-report-2020'
/content/drive/My Drive/Colab Notebooks/DLG/slack_report/slack-report-2020


# ネットワーク分析検討

# リンクがつながるユーザの定義
1. 同じトーク（スレッド）で絡んでいる
  * スレッドに返信"をしていない場合はネットワークが作れない→無視
  * 同じスレッドに返信しているユーザの集計方法→要検討
    * そもそも、"スレッドに返信"は双方向リンクなのか片方向リンクなのか？→とりあえず無向グラフで
    * ３人以上絡んでいる場合のあつかい
1. ＠メンションを飛ばしている
  * 実感値としては、メンションをつけている場合のほうが少ない→１．と比較
  * ＠channelの場合は、誰が対象なのかわからない（各ユーザのチャンネル参加履歴が不明）→ユーザネットワークという点では無視できる？
1. 24時間以内に同一チャンネルに投稿している
  * 同じ話題かどうかの判断が付かないままネットワークを構築
  * 同じ話題について話していても24時間以内とは限らない

### 仮方針：１．と２．のデータを作ってみて比べる？両方必要そうであればマージ

In [111]:
!pip install japanize-matplotlib

In [120]:
import pandas as pd, matplotlib.pyplot as plt, numpy as np
import japanize_matplotlib
from itertools import product, combinations
from datetime import datetime, date

In [121]:
min_link_num = 3
min_degree_num = 2

In [122]:
df_reply = pd.read_csv('reply_table.csv',parse_dates=['reply_ts','thread_ts','target_date'])

In [123]:
df_user = pd.read_csv('user_master.csv')

In [124]:
df_reply.dtypes

channel_id                  object
reply_num                    int64
reply_ts       datetime64[ns, UTC]
reply_id                    object
reply_user                  object
thread_ts      datetime64[ns, UTC]
talk_id                     object
talk_user                   object
target_date         datetime64[ns]
dtype: object

In [125]:
# 各user_idをマトリクスindexに変換
def add_user_mtx_id(arg_df_user, arg_df_reply):
  dict_user_idx = dict(zip(arg_df_user['user_id'], range(len(arg_df_user))))
  for user_type in ['talk', 'reply']:
    arg_df_reply['%s_user_idx' % user_type] = arg_df_reply['%s_user' % user_type].apply(lambda x: dict_user_idx[x])
  return arg_df_reply

In [126]:
# 各replyでの関係ユーザ同士をカウント
# 1スレッド1カウント

# count the related users in df_user from df_reply
def count_related_users(arg_df_user, arg_df_reply):
  total_user_num = len(arg_df_user)
  reply_matrix = np.zeros((total_user_num, total_user_num))

  for thread_id in df_reply['talk_id'].unique():
    df_tmp = arg_df_reply[arg_df_reply['talk_id']==thread_id][['talk_user_idx', 'reply_user_idx']]
    
    arr_users_idx = np.unique(df_tmp.values.flatten())
    for i,j in product(arr_users_idx, arr_users_idx):
      reply_matrix[i,j] += 1
  return reply_matrix


In [127]:
def reset_cross_values(arg_matrix_arr):
  if len(arg_matrix_arr.shape)!=2:
    raise ValueError('input matrix must be 2 Two-dimensional')
  x_size, y_size = arg_matrix_arr.shape
  if x_size != y_size:
    raise ValueError('input matrix must be Square matrix')
  return arg_matrix_arr * (np.ones(x_size) - np.identity(x_size))

In [128]:
def create_df_reply_martix(arg_df_user, arg_df_reply):
  df_reply = add_user_mtx_id(arg_df_user, arg_df_reply)
  reply_matrix = reset_cross_values(count_related_users(arg_df_user, df_reply))
  return pd.DataFrame(reply_matrix, index = df_user['user_id'], columns=df_user['user_id'])

In [129]:
def create_df_reply_lists(arg_df_user, arg_df_reply_matrix):
  # reform reply_matrix to list format
  df_reply_lists = pd.DataFrame(combinations(arg_df_reply_matrix.index,2), columns = ['user1', 'user2'])
  df_reply_lists['count'] = df_reply_lists.apply(lambda r: arg_df_reply_matrix.loc[r['user1'],r['user2']],axis = 1)

  # shape & convert user_name of reply_list
  df_reply_lists.rename(columns={'user1':'user_id'}, inplace = True)
  df_reply_lists = pd.merge(df_reply_lists,arg_df_user,how='left', on = 'user_id')
  df_reply_lists.rename(columns={'name':'user_name1'}, inplace = True)
  df_reply_lists = df_reply_lists[['user_name1', 'user2', 'count']]
  
  df_reply_lists.rename(columns={'user2':'user_id'}, inplace = True)
  df_reply_lists = pd.merge(df_reply_lists,arg_df_user,how='left', on = 'user_id')
  df_reply_lists.rename(columns={'name':'user_name2'}, inplace = True)
  df_reply_lists = df_reply_lists[['user_name1', 'user_name2', 'count']]
  return df_reply_lists

In [130]:
def filter_links_by_reply_counts(arg_df_edges, arg_min_reply_num_link):
  return arg_df_edges[arg_df_edges['count'] >= arg_min_reply_num_link]

In [131]:
def data_shaping_for_visualization(arg_df_user, arg_df_reply_lists):
  active_user_name = np.unique(arg_df_reply_lists[arg_df_reply_lists['count']>0][['user_name1','user_name2']].values.flatten())

  df_user = arg_df_user.copy()
  df_user = df_user[df_user['name'].isin(active_user_name)]  
  # df_user['in_current'] = df_user['target_date'].apply(lambda x : True if x == df_user['target_date'].max() else False)
  
  # distinct users by name 
  df_user = df_user.groupby('name').sum().reset_index()

  df_user['talk_count_range'] = pd.qcut(df_user['talk_count'],5, labels=range(5))
  df_user.sort_values(by='talk_count', ascending=False, inplace = True)
  return df_user

In [132]:
# 全期間のネットワークデータ
df_reply_matrix = create_df_reply_martix(df_user, df_reply)
df_reply_lists = filter_links_by_reply_counts(create_df_reply_lists(df_user, df_reply_matrix), min_link_num) #最低リンク疎通回数の設定

df_user_all = data_shaping_for_visualization(df_user, df_reply_lists)

In [133]:
df_reply_lists.to_csv('reply_network_edges_all.csv', index=False)
df_user_all.to_csv('user_all.csv', index=False)

# ネットワークモデルの構築

In [134]:
import networkx as nx, pandas as pd, matplotlib.pyplot as plt
import japanize_matplotlib


In [135]:
def add_network_analysis_results(arg_df_user, arg_df_edge):
  # create network
  G = nx.Graph()
  for _,r in arg_df_edge.iterrows():
    G.add_edge(r.user_name1, r.user_name2, weight=r.count)
    
  # add network analysis results
  df_user = pd.merge(arg_df_user.copy(), pd.DataFrame(G.degree,columns=['name','degree']),on='name')
  df_user = pd.merge(df_user, pd.DataFrame(nx.degree_centrality(G).items(),columns=['name','degree_centrality']),on='name')
  df_user = pd.merge(df_user, pd.DataFrame(nx.betweenness_centrality(G).items(),columns=['name','betweenness_centrality']),on='name')

  #df_user.index = df_user['name']
  return df_user, G

In [136]:
def filter_links_by_degree(arg_df_user, arg_df_edge, arg_num_of_degree):
  df_user = arg_df_user[arg_df_user['degree'] >= arg_num_of_degree]
  df_edge = arg_df_edge[((arg_df_edge['user_name1'].isin(df_user['name']))) &
                        (arg_df_edge['user_name2'].isin(df_user['name']))]
  
  #フィルター後に再計算
  df_user, _ = add_network_analysis_results(df_user[['name', 'talk_count','talk_count_range']], df_edge)


  return df_user, df_edge

In [137]:
df_user_all, _ = add_network_analysis_results(pd.read_csv('user_all.csv'),
                                              pd.read_csv('reply_network_edges_all.csv'))

df_user_all, df_reply_lists = filter_links_by_degree(df_user_all, df_reply_lists, min_degree_num) # 最低次数の設定


df_reply_lists.to_csv('reply_network_edges_all.csv', index=False,float_format='%.18f')
df_user_all.to_csv('user_all.csv', index=False,float_format='%.18f')

In [138]:
df_user_all.describe()['degree	degree_centrality	betweenness_centrality'.split()]

,degree,degree_centrality,betweenness_centrality
count,92.000000,92.000000,92.000000
mean,13.130435,0.144290,0.009911
std,14.894661,0.163678,0.051919
min,2.000000,0.021978,0.000000
25%,3.000000,0.032967,0.000000
50%,8.000000,0.087912,0.000088
75%,17.250000,0.189560,0.001748
max,89.000000,0.978022,0.491867


# 日付ごとの推移

In [144]:
import pickle
from more_itertools import pairwise
from dateutil import parser
#from datetime import datetime, date

In [145]:
date_span_str = ['2019-{:02d}-01'.format(x) for x in range(6,13,1)]+['2020-{:02d}-01'.format(x) for x in range(1,9,1)]

In [146]:
date_span_str

['2019-06-01',
 '2019-07-01',
 '2019-08-01',
 '2019-09-01',
 '2019-10-01',
 '2019-11-01',
 '2019-12-01',
 '2020-01-01',
 '2020-02-01',
 '2020-03-01',
 '2020-04-01',
 '2020-05-01',
 '2020-06-01',
 '2020-07-01',
 '2020-08-01']

In [147]:
df_reply = pd.read_csv('reply_table.csv',parse_dates=['reply_ts','thread_ts','target_date'])

In [148]:
df_user = pd.read_csv('user_master.csv')

In [149]:
dict_results = dict()
for start, end in pairwise(date_span_str):
  one_result = dict()

  one_result['start_str'] = start
  one_result['end_str'] = end

  one_result['start'] = parser.parse(start)
  one_result['end'] = parser.parse(end)
  print(one_result['start'], one_result['end'])
  one_result['df_reply'] = df_reply.copy()[(one_result['start'] <= df_reply['target_date'])
                                            & (df_reply['target_date'] < one_result['end'])]
  one_result['df_user'] = df_user.copy()
  one_result['df_reply_matrix'] = create_df_reply_martix(one_result['df_user'], 
                                                         one_result['df_reply'])
  one_result['df_edges'] = filter_links_by_reply_counts(create_df_reply_lists(one_result['df_user'], 
                                                        one_result['df_reply_matrix'])
                                                        , min_link_num)  # 最低リプライ回数
  one_result['df_user'] = data_shaping_for_visualization(one_result['df_user'], 
                                                         one_result['df_edges'])

  df_user_tmp, graph_tmp = add_network_analysis_results(one_result['df_user'], 
                                                        one_result['df_edges'])
  
  one_result['df_user'], one_result['df_edges'] = filter_links_by_degree(df_user_tmp, one_result['df_edges'], min_degree_num) # 最低次数の設定

  one_result['graph'] = graph_tmp

  dict_results[(start, end)] = one_result

2019-06-01 00:00:00 2019-07-01 00:00:00
2019-07-01 00:00:00 2019-08-01 00:00:00
2019-08-01 00:00:00 2019-09-01 00:00:00
2019-09-01 00:00:00 2019-10-01 00:00:00
2019-10-01 00:00:00 2019-11-01 00:00:00
2019-11-01 00:00:00 2019-12-01 00:00:00
2019-12-01 00:00:00 2020-01-01 00:00:00
2020-01-01 00:00:00 2020-02-01 00:00:00
2020-02-01 00:00:00 2020-03-01 00:00:00
2020-03-01 00:00:00 2020-04-01 00:00:00
2020-04-01 00:00:00 2020-05-01 00:00:00
2020-05-01 00:00:00 2020-06-01 00:00:00
2020-06-01 00:00:00 2020-07-01 00:00:00
2020-07-01 00:00:00 2020-08-01 00:00:00


In [150]:
dict_results['2019-06-01','2020-08-01'] = dict()
dict_results['2019-06-01','2020-08-01']['df_edges'] = df_reply_lists
dict_results['2019-06-01','2020-08-01']['df_user'] = df_user_all

In [151]:
with open('period_ntw.pkl', 'wb') as f:
    pickle.dump(dict_results, f)

In [152]:
for start, end in pairwise(date_span_str):
  dict_results[start,end]['df_edges'].to_csv('monthly/edges_%s_%s.csv' % (start, end), index=False)
  dict_results[start,end]['df_user'].to_csv('monthly/user_%s_%s.csv' % (start, end), index=False)
